# WIP: Making a ColocatedData object with Pyaerocom
Lewis Blake
lewisb@met.no

This is a tutorial showing how the current version of pyaerocom can be used for creating colocated data objects which minimizes the risk of processing errors. It is a preliminary tutorial, and therefore subject to change as development continues. As the interface becomes better defined, this should simplify the process and changes will be reflected in updated versions of this tutorial. Feedback welcome.

You may want to start by creating a Python virtual environment before begining and installing pyaerocom into it.


```console
python3 -m venv --prompt cams2_82 .venv
source .venv/bin/activate
pip install pyaerocom
pya --help
``` 

In [2]:
import pyaerocom
import xarray as xr
import pandas as pd

Let's say we have done the collocation and stored the results in some object. As an example we will assume this is a netcdf file called `my_colocated_data.nc`.

Our goal is to create a `ColocatedData` object named `od550aer_od550aer_MOD-IFS-OSUITE_REF-AeronetL1.5-d_20130101_20221231_daily_ALL-wMOUNTAINS.nc`

The general format for `ColocateData` objects file name is `"{obs_var_name}_{mod_var_name}_MOD-{model_name}_REF-{obs_name}_{date_start]_{date_end}_{ts_type}_{filter_name}.nc"`

- `obs_var_name`: Aerocom name for the obs variable. Can be found by searching here: https://github.com/metno/pyaerocom/blob/main-dev/pyaerocom/data/variables.ini
- `mod_var_name`: Aerocom name for model variable.
- `model_name`: Name of the model 
- `obs_name`: Name of the observation network
- `date_start`: Start of the timerseries in the colocated array formated YYYYMMDD
- `date_end`: End of the timeseries in the colocated array formated YYYYMMDD
- `ts_type`: Time series type (frequency), for example "daily" or "monthly".
- `filter_name`: name of the region filter applied. If no spatial filering applied, should be the string "ALL-wMOUNTAINS".

Note that the fields provided here in the name **must** match their associated values in the metadata below. For exmaple, the `model_name` here in the name needs to match the `model_name` in the metadata.

A valid (and encouraged) way to create a `ColocatedData` object is to convert it into an xarray DataArray with the correct structure, dimensions, and metadata. You will have you object stored in some format. The task of CAMS2_82 partners will then be to do translation of their colocated data into this format.

Included in pyaerocom is a colocated data object file. In fact there are a few, however there is one specifically included for this tutorial. Pyaerocom can a command line interface which is usefule for this. On the command line there are several functions available.

In [19]:
! pya --help

                                                                                
 Usage: pya [OPTIONS] COMMAND [ARGS]...                                         
                                                                                
 🦄 Pyaerocom Command Line Interface                                            
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --version             -V                                                     │
│ --install-completion            Install completion for the current shell.    │
│ --show-completion               Show completion for the current shell, to    │
│                                 copy it or customize the installation.       │
│ --help                          Show this message and exit.                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ─────────────────

The `getsampledata` command will download the neccessary files into `~/MyPyaerocom`.

In [20]:
!  pya getsampledata

In [25]:
filepath = "~/MyPyaerocom/testdata-minimal/modeldata/colocated_netcdf/my_colocated_data.nc"
data = xr.open_dataarray(filepath)

Note that while included in pyaerocom, this data is not actually CF-compliant. For example, latitude and longitude have units degrees, but really should be "degrees_north" and "degrees_east" respectively. Let's fix that.

In [34]:
data.latitude.attrs["units"] = "degrees_north"
data.longitude.attrs["units"] = "degrees_east"

In the current design, such A `ColocatedData` object comprises 3 or 4 dimensions.

- The first dimension (`data_source`, index 0) is ALWAYS length 2 and specifies the two datasets that were co-located (index 0 is obs, index 1 is model). 
- The second dimension is `time`. 
- 3rd dimension is `station_name` in case of 3D colocated data. For 4D colocated data the 3rd and 4th dimension are latitude and longitude, respectively.

3D colocated data is typically created when a model is colocated with station based ground based observations. 4D colocated data is created when a model is colocated with another model or satellite observations, that cover large parts of Earth's surface (other than discrete lat/lon pairs in the case of ground based station locations).

We may be curious about the valid potential options available to us. We can check the fields in a few relevant classes to determine what these options are and to check which data types they are allowed to be.

The documentation for the `ColocationSetup` class can be found here: https://pyaerocom.readthedocs.io/en/latest/api.html#pyaerocom.colocation.colocation_setup.ColocationSetup

In [4]:
from pyaerocom import ColocationSetup, ColocatedData

In [35]:
ColocationSetup.model_fields

{'model_id': FieldInfo(annotation=Union[str, NoneType], required=True),
 'obs_id': FieldInfo(annotation=Union[str, NoneType], required=True),
 'obs_vars': FieldInfo(annotation=Union[tuple[str, ...], str], required=True),
 'ts_type': FieldInfo(annotation=str, required=True),
 'start': FieldInfo(annotation=Union[Timestamp, int, str, NoneType], required=True),
 'stop': FieldInfo(annotation=Union[Timestamp, int, str, NoneType], required=True),
 'obs_config': FieldInfo(annotation=Union[PyaroConfig, NoneType], required=False, default=None),
 'OBS_VERT_TYPES_ALT': FieldInfo(annotation=dict[str, str], required=False, default={'Surface': 'ModelLevel', '2D': '2D'}),
 'CRASH_ON_INVALID': FieldInfo(annotation=bool, required=False, default=False),
 'FORBIDDEN_KEYS': FieldInfo(annotation=list[str], required=False, default=['var_outlier_ranges', 'var_ref_outlier_ranges', 'remove_outliers']),
 'filter_name': FieldInfo(annotation=str, required=False, default='ALL-wMOUNTAINS'),
 'basedir_coldata': Field

The documentation for the `ColocatedData` object can be found here: https://pyaerocom.readthedocs.io/en/latest/api.html#pyaerocom.colocation.colocated_data.ColocatedData

In [6]:
ColocatedData.model_fields

{'data': FieldInfo(annotation=Union[Path, str, DataArray, ndarray, NoneType], required=False, default=None)}

Let's build this `ColocatedData` oibject by hand. We start with the array: `arr`, assuming its dimension matches the structure described above.

In [7]:
arr = data.data

In [8]:
arr.shape

(2, 3652, 665)

Now the coordinates...

In [9]:
data_source = ['AeronetSunV3Lev1.5.daily', 'ECMWF_OSUITE']
time = pd.date_range(start="2013-01-01", end="2022-12-31", freq = "D") # daily frequency
station_names = list(data.coords["station_name"].data)

In [10]:
lats = data.latitude.data
lons = data.longitude.data
alts = data.altitude.data

In [13]:
coords = {
    "data_source": data_source,
    "time": time,
    "station_name": station_names,
    "latitude": ("station_name", lats),
    "longitude": ("station_name", lons),
    "altitude": ("station_name", alts),
}

Here we cheat a bit with the station names as it is a very long list, so we get it directly from the `data` object, but in principle this is a list of string that you have somewhere. We also cheat a bit with the latitudes, longitudes, and altitudes, but again these are arrays that you have somewhere.

In order to ensure safe processing, we require **all** the following metadata fields. Here is an example of a dictionary containing some example values.

In [14]:
metadata = {
    "obs_vars": "od550aer", # must match boths obs_var_name and mod_var_name in filename
    "ts_type": "daily",
    "filter_name": "ALL-wMOUNTAINS",
    "ts_type_src": ["hourly", "hourly"], # must be a list of strings, a ts type for each data_source
    "var_units": ["1", "1"], # must have units for each data_source
    "data_level": 3,
    "revision_ref": "20221027",
    "from_files": ['ECMWF_OSUITE.daily.od550aer.2013.nc', 'ECMWF_OSUITE.daily.od550aer.2014.nc', 'ECMWF_OSUITE.daily.od550aer.2015.nc', 'ECMWF_OSUITE.daily.od550aer.2016.nc', 'ECMWF_OSUITE.daily.od550aer.2017.nc', 'ECMWF_OSUITE.daily.od550aer.2018.nc', 'ECMWF_OSUITE.daily.od550aer.2019.nc', 'ECMWF_OSUITE.daily.od550aer.2020.nc', 'ECMWF_OSUITE.daily.od550aer.2021.nc', 'ECMWF_OSUITE.daily.od550aer.2022.nc'],
    "from_files_ref": "None",
    "colocate_time": 0, # default is 0
    "obs_is_clim": 0, # default is 0
    "pyaerocom": pyaerocom.__version__,
    "CONV!min_num_obs": str(dict(monthly=dict(daily=3))), # yes, needed
    "resample_how": "mean", # default
    "obs_name": "AeronetL1.5-d", # must match obs_name in filename
    "vert_code": "Column", # Literal["Column", "Profile", "Surface", "ModelLevel"]. default is "Surface"
    "diurnal_only": 0, # default is 0
    "zeros_to_nan": 1, # default is 0
}


We can check that we have the right data types by passing this metadata object to Pyaerocom's `ColocationSetup` class. Under the hood Pyaerocom has checked that we provided the correct data types.

In [42]:
col_stp = ColocationSetup(**metadata)

Now we put it all in an xarray DataArray...

In [43]:
da = xr.DataArray(
    data = arr,
    dims = ["data_source", "time", "station_name"],
    coords = coords,
    attrs = metadata
)
da.name = "od550aer" # Important to give the DataArray the name of the variable.

And we put that DataArray into Pyaerocom `ColocatedData`...

In [45]:
coldata = ColocatedData(da)

Under the hood Pyaerocom has checked that we have the correct dimensions on this object.

We can save this ColocatedData object.

In [46]:
savename = "od550aer_od550aer_MOD-IFS-OSUITE_REF-AeronetL1.5-d_20130101_20221231_daily_ALL-wMOUNTAINS.nc"
out_dir = "/home/lewisb/Documents/CAMS2_82/coldata"

In [47]:
coldata.to_netcdf(out_dir=out_dir, savename=savename)

'/home/lewisb/Documents/CAMS2_82/coldata/od550aer_od550aer_MOD-IFS-OSUITE_REF-AeronetL1.5-d_20130101_20221231_daily_ALL-wMOUNTAINS.nc'

When everything is done we should we able to do a `ncdump -h` and verify its contents.


```console

ncdump -h od550aer_od550aer_MOD-IFS-OSUITE_REF-AeronetL1.5-d_20130101_20221231_daily_ALL-wMOUNTAINS.nc
netcdf od550aer_od550aer_MOD-IFS-OSUITE_REF-AeronetL1.5-d_20130101_20221231_daily_ALL-wMOUNTAINS {
dimensions:
	data_source = 2 ;
	time = 3652 ;
	station_name = 665 ;
variables:
	string data_source(data_source) ;
	int64 time(time) ;
		time:units = "days since 2013-01-01 12:00:00" ;
		time:calendar = "proleptic_gregorian" ;
	string station_name(station_name) ;
	double latitude(station_name) ;
		latitude:_FillValue = NaN ;
		latitude:standard_name = "latitude" ;
		latitude:units = "degrees_north" ;
	double longitude(station_name) ;
		longitude:_FillValue = NaN ;
		longitude:standard_name = "longitude" ;
		longitude:units = "degrees_east" ;
	double altitude(station_name) ;
		altitude:_FillValue = NaN ;
	double od550aer(data_source, time, station_name) ;
		od550aer:_FillValue = NaN ;
		string od550aer:data_source = "AeronetSunV3Lev1.5.daily", "ECMWF_OSUITE" ;
		string od550aer:var_name = "od550aer", "od550aer" ;
		string od550aer:var_name_input = "od550aer", "od550aer" ;
		od550aer:ts_type = "daily" ;
		od550aer:filter_name = "ALL-wMOUNTAINS" ;
		string od550aer:ts_type_src = "daily", "daily" ;
		string od550aer:var_units = "1", "1" ;
		od550aer:data_level = 3LL ;
		od550aer:revision_ref = "20221027" ;
		string od550aer:from_files = "ECMWF_OSUITE.daily.od550aer.2013.nc", "ECMWF_OSUITE.daily.od550aer.2014.nc", "ECMWF_OSUITE.daily.od550aer.2015.nc", "ECMWF_OSUITE.daily.od550aer.2016.nc", "ECMWF_OSUITE.daily.od550aer.2017.nc", "ECMWF_OSUITE.daily.od550aer.2018.nc", "ECMWF_OSUITE.daily.od550aer.2019.nc", "ECMWF_OSUITE.daily.od550aer.2020.nc", "ECMWF_OSUITE.daily.od550aer.2021.nc", "ECMWF_OSUITE.daily.od550aer.2022.nc" ;
		od550aer:from_files_ref = "None" ;
		od550aer:colocate_time = 0LL ;
		od550aer:obs_is_clim = 0LL ;
		od550aer:pyaerocom = "0.13.2.dev0" ;
		od550aer:CONV\!min_num_obs = "{\'monthly\': {\'daily\': 3}}" ;
		od550aer:resample_how = "mean" ;
		od550aer:model_name = "IFS-OSUITE" ;
		od550aer:obs_name = "AeronetL1.5-d" ;
		od550aer:vert_code = "Column" ;
		od550aer:diurnal_only = 0LL ;
		od550aer:coordinates = "altitude longitude latitude" ;
```